In [2]:
import numpy as np
import json
import time
import cv2
from glob import glob

from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

# 1. 초기 R, t Load

In [3]:
with open("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/ee2cam.json", "r") as f:
    data = json.load(f)

R_ee2cam = np.array(data["R_ee2cam"])
t_ee2cam = np.array(data["t_ee2cam"]).reshape(3)
q_init = R.from_matrix(R_ee2cam).as_quat()
x0 = np.hstack((q_init, t_ee2cam))

## 2. pose 및 charuco load

In [5]:
sorted_pose_files = sorted(glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/poses/*.json"))
sorted_charuco_files = sorted(glob("/home/ros/llm_robot/data/Calibration/Eye-in-Hand2/*.json"))

In [6]:
len(sorted_pose_files), len(sorted_charuco_files[:-1])

(20, 20)

In [7]:
pose_data = []
for pf, cf in zip(sorted_pose_files, sorted_charuco_files[:-1]):
    with open(pf) as f: pose = json.load(f)
    with open(cf) as f: charuco = json.load(f)
    
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3)
    R_board2cam = R.from_rotvec(np.array(charuco["rvec"]).reshape(3)).as_matrix()
    t_board2cam = np.array(charuco["tvec"]).reshape(3)
    
    pose_data.append({
        "R_base2ee": R_base2ee,
        "t_base2ee": t_base2ee,
        "R_board2cam": R_board2cam,
        "t_board2cam": t_board2cam
    })

# 3. 최적화 함수 

In [8]:
def residuals(x, data):
    q = x[:4]
    t = x[4:]
    R_ee2cam = R.from_quat(q).as_matrix()
    t_ee2cam = t
    res = []
    for d in data:
        R_pred = d["R_base2ee"] @ R_ee2cam
        t_pred = d["R_base2ee"] @ t_ee2cam + d["t_base2ee"]
        R_err = R.from_matrix(R_pred.T @ d["R_board2cam"]).as_rotvec()
        t_err = t_pred - d["t_board2cam"]
        res.extend(R_err)
        res.extend(t_err)
    res.append(np.linalg.norm(q) - 1)
    return np.array(res)

result = least_squares(residuals, x0, args=(pose_data,), method='lm')

In [9]:
result

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [-4.781e-02 -2.569e-01 ...  3.513e+01 -1.981e-08]
           x: [-6.669e-01  7.422e-01 -5.967e-02  3.090e-02 -5.008e+01
                1.056e+02  2.437e+02]
        cost: 44955.454108226535
         jac: [[-2.520e-01 -5.261e-02 ...  0.000e+00  0.000e+00]
               [-8.418e-02 -2.964e-02 ...  0.000e+00  0.000e+00]
               ...
               [ 0.000e+00  0.000e+00 ... -1.712e-02 -9.714e-01]
               [-6.669e-01  7.422e-01 ...  0.000e+00  0.000e+00]]
        grad: [ 2.444e-05  2.072e-05 -1.126e-05  7.230e-06 -3.440e-06
                6.561e-09  1.015e-06]
  optimality: 2.4435802324824494e-05
 active_mask: [0 0 0 0 0 0 0]
        nfev: 49
        njev: None

# 결과 저장

In [ ]:
q_opt = result.x[:4] / np.linalg.norm(result.x[:4])
t_opt = result.x[4:]
R_opt = R.from_quat(q_opt).as_matrix()

result_dict = {
    "R_cam2ee": R_opt.tolist(),
    "t_cam2ee": (t_opt / 1000).tolist()
}
with open("optimized_cam2ee_LM.json", "w") as f:
    json.dump(result_dict, f, indent=2)
